In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree

from src.utils.models_pkl import load_pickle

In [ ]:
from config.config_data import DATA_PATH

In [ ]:
MODEL_PATH = Path("../models") / "XGB.pkl"
model = load_pickle(MODEL_PATH)

In [ ]:
X_train = pd.read_csv(".." / Path(DATA_PATH.parent / "train_X.csv"))
y_train = pd.read_csv(".." / Path(DATA_PATH.parent / "train_y.csv"))
X_test = pd.read_csv(".." / Path(DATA_PATH.parent / "test_X.csv"))
y_test = pd.read_csv(".." / Path(DATA_PATH.parent / "test_y.csv"))

In [ ]:
# Train the XGBoost model
model.fit(X_train, y_train)

#### Linear Regression as a Surrogate Model

In [ ]:
# Train a Linear Regression as a surrogate model to approximate XGBoost predictions
linear_reggression_surrogate = LinearRegression()
linear_reggression_surrogate.fit(X_train, model.predict(X_train))

In [ ]:
from sklearn.linear_model import LassoCV

lasso = LassoCV(alphas=[0.01, 0.1, 1.0, 10.0, 100.0], cv=5)
lasso.fit(X_train, y_train)

# Get the optimal alpha value
optimal_alpha = lasso.alpha_
print(optimal_alpha)

linear_regression_surrogate_lasso = LinearRegression()
linear_regression_surrogate_lasso.fit(X_train, y_train)
lasso_coefficients = lasso.coef_

In [ ]:
# Evaluate the surrogate model's performance
surrogate_predictions = linear_reggression_surrogate.predict(X_train)
lasso_surrogate_predictions = linear_regression_surrogate_lasso.predict(X_train)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(model.predict(X_train), surrogate_predictions))
lasso_rmse = np.sqrt(mean_squared_error(model.predict(X_train), lasso_surrogate_predictions))

print("Linear Regression Surrogate Model RMSE: {:.4f}".format(rmse))
print("Lasso Linear Regression Surrogate Model RMSE: {:.4f}".format(lasso_rmse))

In [ ]:
# Get the coefficients of the surrogate model
coef = linear_reggression_surrogate.coef_

# Get feature names (assuming X_train is a DataFrame)
feature_names = X_train.columns

In [ ]:
# Order by the absolute values of coefficients
importance_df = pd.DataFrame({"Feature Name": feature_names, "Coefficients": coef})

importance_df["Absolute Coefficients"] = importance_df["Coefficients"].abs()
importance_df = importance_df.sort_values(by="Absolute Coefficients", ascending=False)
importance_df = importance_df.drop(columns="Absolute Coefficients")
pd.options.display.float_format = "{:.2f}".format

print("Feature Importances (Ordered by Absolute Coefficient Value):")
print(importance_df)

In [ ]:
# Plot feature importance
plt.figure(figsize=(10, 8))
plt.barh(feature_names, coef, color="green")
plt.xlabel("Coefficient Magnitude")
plt.ylabel("Feature")
plt.title("Feature Importance in Linear Regression Surrogate Model")
plt.show()

#### Decision Tree as a Surrogate Model

In [ ]:
# Define a range of maximum depths to test
max_depths = list(range(2, 30))
cv_scores = []

# Iterate over different max_depth values and perform cross-validation
for max_depth in max_depths:
    decision_tree_surrogate = DecisionTreeClassifier(max_depth=max_depth)
    scores = cross_val_score(decision_tree_surrogate, X_train, y_train, cv=5, scoring="accuracy")
    mean_accuracy = np.mean(scores)
    cv_scores.append(mean_accuracy)

# Plot the mean cross-validation scores vs. max depth
plt.figure(figsize=(10, 3))
plt.plot(max_depths, cv_scores, color="green")
plt.xlabel("Max Depth")
plt.ylabel("Mean Cross-Validation Accuracy")
plt.title("Surrogate Model Performance vs. Max Depth (Cross-Validation)")
plt.grid(True)
plt.show()

In [ ]:
# Train a Decision Tree as a surrogate model to approximate XGBoost predictions
max_depth = 5

decision_tree_surrogate = DecisionTreeClassifier(max_depth=max_depth)
decision_tree_surrogate.fit(X_train, model.predict(X_train))

In [ ]:
# Evaluate the surrogate model's performance
decision_tree_surrogate_preds = decision_tree_surrogate.predict(X_test)

surrogate_accuracy = accuracy_score(y_test, decision_tree_surrogate_preds)
print("Decision Tree Surrogate Model Accuracy: {:.2f}%".format(surrogate_accuracy * 100))

In [ ]:
# Decision Tree Visualization
plt.figure(figsize=(12, 6))
plot_tree(
    decision_tree_surrogate,
    filled=True,
    feature_names=X_train.columns,
    class_names=["Warning", "Citation"],
)
plt.show()

In [ ]:
feature_importance = decision_tree_surrogate.feature_importances_
feature_names = X_train.columns

# Sort feature importances in descending order
sorted_idx = feature_importance.argsort()

In [ ]:
# Feature Importance Plot

plt.figure(figsize=(8, 10))
plt.barh(range(X_train.shape[1]), feature_importance[sorted_idx], color="green")
plt.yticks(range(X_train.shape[1]), [feature_names[i] for i in sorted_idx])
plt.ylabel("Feature")
plt.xlabel("Feature Importance")
plt.title("Feature Importance Analysis")
plt.show()